In [105]:
import pandas as pd
import mplfinance as mpf
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from datetime import timedelta

# -----------------------------
# Step 1: Load CSV
# -----------------------------
file_path = r"C:\Users\Lenovo\Downloads\FX_EURUSD, 5_71985.csv"
df = pd.read_csv(file_path, encoding='utf-8-sig')
df.columns = df.columns.str.strip()

# Timestamp column
timestamp_col = next((c for c in ['CC','time','timestamp'] if c in df.columns), None)
df['timestamp'] = pd.to_datetime(df[timestamp_col], unit='s')

# Convert timestamp to IST (UTC+5:30)
df['timestamp'] = df['timestamp'] + timedelta(hours=5, minutes=30)

df = df.sort_values('timestamp').reset_index(drop=True)

# -----------------------------
# Step 2: Candle detection
# -----------------------------
df['bullish'] = df['close'] > df['open']
df['bearish'] = df['close'] < df['open']
df['doji'] = df['close'] == df['open']

# -----------------------------
# Step 3: Strategy Detection
# -----------------------------
trade_signals = []
i = 0
while i < len(df):
    if df.loc[i,'doji']:
        i += 1
        continue  # doji resets streak

    # Detect streak
    streak_color = None
    streak_len = 0
    streak_indices = []
    for j in range(i, len(df)):
        if df.loc[j,'doji']:
            break  # reset streak on doji
        candle_color = 'bullish' if df.loc[j,'bullish'] else 'bearish' if df.loc[j,'bearish'] else None
        if streak_color is None and candle_color is not None:
            streak_color = candle_color
        if candle_color == streak_color:
            streak_len += 1
            streak_indices.append(j)
        else:
            break

    if streak_len >= 4:
        streak_end = streak_indices[-1]
        last_open = df.loc[streak_end,'open']
        last_high = df.loc[streak_end,'high']
        last_low = df.loc[streak_end,'low']
        # Handle degenerate target if open = high/low
        if streak_color=='bullish' and last_low == last_open:
            if len(streak_indices) >= 2:
                last_low = df.loc[streak_indices[-2],'low']
        if streak_color=='bearish' and last_high == last_open:
            if len(streak_indices) >= 2:
                last_high = df.loc[streak_indices[-2],'high']

        # Pullback candles (1-2 opposite candles)
        pullback_indices = []
        pullback_allowed = 2
        valid_streak = True
        breaking_candle_index = None
        for k in range(streak_end+1, len(df)-1):
            if df.loc[k,'doji']:
                valid_streak = False
                break  # doji in pullback resets streak
            candle_color = 'bullish' if df.loc[k,'bullish'] else 'bearish'
            if candle_color != streak_color:
                # Must not cross streak open
                if streak_color=='bullish' and df.loc[k,'low'] <= last_open:
                    valid_streak = False
                    break
                if streak_color=='bearish' and df.loc[k,'high'] >= last_open:
                    valid_streak = False
                    break
                pullback_indices.append(k)
                if len(pullback_indices) >= pullback_allowed:
                    continue
            else:
                breaking_candle_index = k
                break

        if not valid_streak or len(pullback_indices) > pullback_allowed:
            i = streak_end + 1
            continue

        # -----------------------------
        # Step 3a: Target calculation
        # -----------------------------
        target = None
        if streak_color=='bullish':
            fifty_percent = last_high - (last_high - last_low)/2
            touched_50 = any(df.loc[idx,'high'] >= fifty_percent for idx in pullback_indices)
            if touched_50 or len(pullback_indices)==0:
                target = last_low
            else:
                target = min(df.loc[idx,'low'] for idx in pullback_indices)
        else:  # bearish streak
            fifty_percent = last_low + (last_high - last_low)/2
            touched_50 = any(df.loc[idx,'low'] <= fifty_percent for idx in pullback_indices)
            if touched_50 or len(pullback_indices)==0:
                target = last_high
            else:
                target = max(df.loc[idx,'high'] for idx in pullback_indices)

        # -----------------------------
        # Step 3b: Find breakout and next candle confirmation
        # -----------------------------
        for k in range(streak_end+1, len(df)-1):
            if df.loc[k,'doji']:
                continue
            if (streak_color=='bullish' and df.loc[k,'close'] < target) or \
               (streak_color=='bearish' and df.loc[k,'close'] > target):
                continue  # target not reached
            next_candle_index = k+1
            if df.loc[next_candle_index,'doji']:
                continue
            next_color = 'bullish' if df.loc[next_candle_index,'bullish'] else 'bearish'
            if (streak_color=='bullish' and next_color=='bullish') or \
               (streak_color=='bearish' and next_color=='bearish'):
                signal_type = 'CALL' if streak_color=='bullish' else 'PUT'
                trade_signals.append({
                    'streak_start': i,
                    'streak_end': streak_end,
                    'breaking_candle': k,
                    'pullback_indices': pullback_indices,
                    'streak_color': streak_color,
                    'target': target,
                    'signal': signal_type,
                    'trade_candle': next_candle_index
                })
            break

        i = streak_end + 1
    else:
        i += 1

# -----------------------------
# Step 4: Trade Simulation
# -----------------------------
trade_log = []
for t in trade_signals:
    idx = t['trade_candle']
    prev_close = df.loc[idx-1,'close']
    next_close = df.loc[idx,'close']
    if (t['signal']=='CALL' and next_close > prev_close) or (t['signal']=='PUT' and next_close < prev_close):
        t['result'] = 'WIN'
    else:
        t['result'] = 'LOSS'
    trade_log.append(t)

# -----------------------------
# Step 5: Save all trades to single PDF with date & time in IST
# -----------------------------
pdf_path = r"C:\Users\Lenovo\Downloads\All_Trades_TargetFixed_IST.pdf"
window_after_trade = 3

with PdfPages(pdf_path) as pdf:
    for t_num, t in enumerate(trade_log, start=1):
        start = t['streak_start']
        end = min(len(df), t['trade_candle'] + window_after_trade + 1)
        plot_df = df.iloc[start:end].copy()
        plot_df.set_index('timestamp', inplace=True)  # DatetimeIndex

        mc = mpf.make_marketcolors(up='green', down='red', edge='black', wick='black', volume='gray')
        s = mpf.make_mpf_style(marketcolors=mc)

        fig, axlist = mpf.plot(plot_df, type='candle', style=s, figsize=(12,6),
                               returnfig=True,
                               hlines=dict(hlines=[t['target']], colors=['orange'], linestyle='dashed'),
                               tight_layout=True)
        ax = axlist[0]

        # Format x-axis to show date + time in IST
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
        plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

        # Highlight pullbacks
        for idx_pb in t['pullback_indices']:
            if idx_pb in plot_df.index:
                ax.axvline(plot_df.index.get_loc(idx_pb), color='blue', linewidth=2)

        # Highlight breaking candle
        if t['breaking_candle'] in plot_df.index:
            ax.axvline(plot_df.index.get_loc(t['breaking_candle']), color='purple', linewidth=2)

        # Highlight trade candle
        if t['trade_candle'] in plot_df.index:
            ax.axvline(plot_df.index.get_loc(t['trade_candle']), color='cyan', linewidth=2)

        # Add trade detection date & time (IST) at top-right
        trade_time = df.loc[t['breaking_candle'], 'timestamp']
        ax.text(0.99, 0.95, f"Trade detected: {trade_time.strftime('%Y-%m-%d %H:%M IST')}",
                transform=ax.transAxes, ha='right', va='top', fontsize=12,
                bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.4))

        fig.suptitle(f'Trade {t_num}: {t["signal"]} ({t["result"]})', fontsize=14)
        pdf.savefig(fig)
        plt.close(fig)

print(f"All trades saved to PDF: {pdf_path}")

# -----------------------------
# Step 6: Trade Summary
# -----------------------------
total_trades = len(trade_log)
wins = len([t for t in trade_log if t['result']=='WIN'])
losses = len([t for t in trade_log if t['result']=='LOSS'])
win_rate = (wins/total_trades*100) if total_trades>0 else 0

print("---------- TRADE SUMMARY ----------")
print(f"Total Trades: {total_trades}, Wins: {wins}, Losses: {losses}, Win Rate: {win_rate:.2f}%")


All trades saved to PDF: C:\Users\Lenovo\Downloads\All_Trades_TargetFixed_IST.pdf
---------- TRADE SUMMARY ----------
Total Trades: 167, Wins: 167, Losses: 0, Win Rate: 100.00%
